In [1]:
! pip install glove_python_binary | grep satistfied

Pretrained Glove file(e.g., glove.6B.txt)을 써서 유사도를 구하려 했으나 discourse_text 단어가 없는 경우가 많았다. <br>
따라서, Feedback Prize text 단어들로 glove를 새로 학습하는 방법을 채택하였다.

### 1. Text 준비
학습할 코퍼스 문서를 단어단위로 쪼개어 리스트를 만든다.

In [4]:

import os
from tqdm import tqdm
import pandas as pd
BASE_DIR = "../input/feedback-prize-2021/"
TRAIN_DIR = BASE_DIR + 'train'

def open_txt():
    train_names, train_texts = [], []
    for f in tqdm(list(os.listdir(TRAIN_DIR))):
        train_names.append(f.replace('.txt', ''))
        train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r', encoding='utf-8').read())
    train_text_df = pd.DataFrame({'id': train_names, 'text': train_texts})

    return train_text_df

train_text_df = open_txt()

100%|██████████| 15594/15594 [00:00<00:00, 115903.10it/s]


In [5]:
doc_list = []
for line in train_text_df['text'].values:
    doc_list.append(line.split())

print("document count :", len(doc_list))

document count : 15594


### 2. Corpus 학습 
Corpus를 통해 데이터를 가공하여 동시 등장 행렬을 생성한다. 이를 기반으로 glove 모델을 학습시킬 것이다.

In [12]:
from glove import Corpus, Glove

corpus = Corpus()
corpus.fit(doc_list, window=3)


### 3. GloVe 학습
생성한 corpus로 glove 학습

In [23]:
# 경사하강법 학습률 0.05, 아웃풋 벡터의 차원 100
glove = Glove(no_components=100, learning_rate=0.01)
# 쓰레드 개수는 4개, 훈련 횟수는 20번, verbose (설명) True
glove.fit(corpus.matrix, epochs=20, no_threads=100, verbose=True)
# 유사도 검색을 위한 행렬의 index 정보 입력

Performing 20 training epochs with 100 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


### 4. Similiarty 계산

In [33]:
glove.add_dictionary(corpus.dictionary)
model_result1 = glove.most_similar("will")
print(model_result1)


[('could', 0.9955123279470905), ('allow', 0.9955064526248815), ('helps', 0.9944355517393171), ('give', 0.9941213214217731)]


### 5. model 저장

In [34]:
glove.save('glove.model')

### 6. model 불러오기

In [3]:
from glove import Corpus, Glove
glove = Glove.load('glove.model')

model_result1 = glove.most_similar("if")
print(model_result1)

[('u', 0.7895759901533169), ('What', 0.7739260268105668), ('there', 0.7594694963223717), ('maybe', 0.759107841133347)]
